# HuggingFace Embeddings

- `Hugging Face` cung cấp một loạt các **mô hình embedding** miễn phí, cho phép thực hiện nhiều tác vụ `embedding` khác nhau một cách dễ dàng.
- Trong hướng dẫn này, chúng ta sẽ sử dụng `langchain_huggingface` để xây dựng một **hệ thống tìm kiếm đơn giản dựa trên embedding văn bản**.
- Các mô hình sau sẽ được sử dụng cho **Text Embedding**:

    - 1️⃣ **multilingual-e5-large-instruct**: Một mô hình `embedding` đa ngôn ngữ dựa trên hướng dẫn.
    - 2️⃣ **multilingual-e5-large**: Một mô hình `embedding` đa ngôn ngữ mạnh mẽ.
    - 3️⃣ **bge-m3**: Được tối ưu hóa cho xử lý văn bản quy mô lớn.


![](https://raw.githubusercontent.com/aidino/LangChain-OpenTutorial/a589b7f082f5b0a358921d3e6e9a0c8d97978eb8/08-Embedding/assets/03-huggingfaceembeddings-workflow.png)

### 🛠️ **Các cấu hình sau sẽ được thiết lập**

-   **Cài đặt đầu ra Jupyter Notebook**
    -   Hiển thị thông báo lỗi chuẩn (`stderr`) trực tiếp thay vì ghi lại chúng.
-   **Cài đặt các gói cần thiết**
    -   Đảm bảo tất cả các phụ thuộc cần thiết đã được cài đặt.
-   **Thiết lập khóa API**
    -   Định cấu hình khóa API để xác thực.
-   **Thiết lập lựa chọn thiết bị PyTorch**
    -   Tự động chọn thiết bị tính toán tối ưu (CPU, CUDA hoặc MPS).
        -   `{"device": "mps"}`: Thực hiện tính toán `embedding` bằng **MPS** thay vì GPU. (Dành cho người dùng Mac)
        -   `{"device": "cuda"}`: Thực hiện tính toán `embedding` bằng **GPU**. (Dành cho người dùng Linux và Windows, yêu cầu cài đặt CUDA)
        -   `{"device": "cpu"}`: Thực hiện tính toán `embedding` bằng **CPU**. (Khả dụng cho tất cả người dùng)
-   **Đường dẫn lưu trữ cục bộ mô hình Embedding**
    -   Xác định đường dẫn cục bộ để lưu trữ các mô hình `embedding`.


In [1]:
# Automatically select the appropriate device
import torch
import platform


def get_device():
    if platform.system() == "Darwin":  # macOS specific
        if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
            print("✅ Using MPS (Metal Performance Shaders) on macOS")
            return "mps"
    if torch.cuda.is_available():
        print("✅ Using CUDA (NVIDIA GPU)")
        return "cuda"
    else:
        print("✅ Using CPU")
        return "cpu"


# Set the device
device = get_device()
print("🖥️ Current device in use:", device)

✅ Using CPU
🖥️ Current device in use: cpu


In [2]:
# Embedding Model Local Storage Path
import os
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

# Set the download path to ./cache/
os.environ["HF_HOME"] = "./cache/"

## Data Preparation for Embedding-Based Search Tutorial

To perform **embedding-based search,** we prepare both a **Query** and **Documents.**  

1. Query  
- Write a **key question** that will serve as the basis for the search.  

In [3]:
# Query
q = "Please tell me more about LangChain."

2. Documents  
- Prepare **multiple documents (texts)** that will serve as the target for the search.  
- Each document will be **embedded** to enable semantic search capabilities.  

In [4]:
# Documents for Text Embedding
docs = [
    "Hi, nice to meet you.",
    "LangChain simplifies the process of building applications with large language models.",
    "The LangChain English tutorial is structured based on LangChain's official documentation, cookbook, and various practical examples to help users utilize LangChain more easily and effectively.",
    "LangChain simplifies the process of building applications with large-scale language models.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]

## Which Text Embedding Model Should You Use?
- Leverage the **MTEB leaderboard** and **free embedding models** to confidently select and utilize the **best-performing text embedding models** for your projects! 🚀  

---

### 🚀 **What is MTEB (Massive Text Embedding Benchmark)?**  
- **MTEB** is a benchmark designed to **systematically and objectively evaluate** the performance of text embedding models.  
    - **Purpose:** To **fairly compare** the performance of embedding models.  
    - **Evaluation Tasks:** Includes tasks like **Classification,**  **Retrieval,**  **Clustering,**  and **Semantic Similarity.**  
    - **Supported Models:** A wide range of **text embedding models available on Hugging Face.**  
    - **Results:** Displayed as **scores,**  with top-performing models ranked on the **leaderboard.**  

🔗 [ **MTEB Leaderboard (Hugging Face)** ](https://huggingface.co/spaces/mteb/leaderboard)  

---

### 🛠️ **Models Used in This Tutorial**  

| **Embedding Model** | **Description** |
|----------|----------|
| 1️⃣ **multilingual-e5-large-instruct** | Offers strong multilingual support with consistent results. |
| 2️⃣ **multilingual-e5-large** | A powerful multilingual embedding model. |
| 3️⃣ **bge-m3** | Optimized for large-scale text processing, excelling in retrieval and semantic similarity tasks. |

1️⃣ **multilingual-e5-large-instruct**
![](./assets/03-huggingfaceembeddings-leaderboard-01.png)

2️⃣ **multilingual-e5-large**
![](./assets/03-huggingfaceembeddings-leaderboard-02.png)

3️⃣ **bge-m3**
![](./assets/03-huggingfaceembeddings-leaderboard-03.png)

## Tính toán độ tương đồng - Similarity Calculation

**Tính toán độ tương đồng bằng tích vô hướng vector**

-   Độ tương đồng được xác định bằng **tích vô hướng** của các vector.

-   **Công thức tính toán độ tương đồng:**

$$ \text{similarities} = \mathbf{query} \cdot \mathbf{documents}^T $$

---

### 📐 **Ý nghĩa toán học của tích vô hướng vector**

**Định nghĩa tích vô hướng vector**

**Tích vô hướng** của hai vector, $\mathbf{a}$ và $\mathbf{b}$, được định nghĩa toán học như sau:

$$ \mathbf{a} \cdot \mathbf{b} = \sum_{i=1}^{n} a_i b_i $$

---

**Mối quan hệ với độ tương đồng cosine**

**Tích vô hướng** cũng liên quan đến **độ tương đồng cosine** và tuân theo tính chất sau:

$$ \mathbf{a} \cdot \mathbf{b} = \|\mathbf{a}\| \|\mathbf{b}\| \cos \theta $$

Trong đó:

-   $\|\mathbf{a}\|$ và $\|\mathbf{b}\|$ đại diện cho **độ lớn** (**chuẩn**, cụ thể là chuẩn Euclidean) của các vector $\mathbf{a}$ và $\mathbf{b}$.
-   $\theta$ là **góc giữa hai vector**.
-   $\cos \theta$ đại diện cho **độ tương đồng cosine** giữa hai vector.

---

**🔍 Giải thích tích vô hướng vector trong độ tương đồng**

Khi **giá trị tích vô hướng lớn** (giá trị dương lớn):

-   **Độ lớn** ($\|\mathbf{a}\|$ và $\|\mathbf{b}\|$) của hai vector lớn.
-   **Góc** ($\theta$) giữa hai vector nhỏ (**$\cos \theta$ tiến gần 1**).

Điều này cho thấy hai vector chỉ theo **hướng tương tự** và **tương đồng về ngữ nghĩa hơn**, đặc biệt khi độ lớn của chúng cũng lớn.

---

### 📏 **Tính toán độ lớn vector (chuẩn)**

**Định nghĩa chuẩn Euclidean**

Đối với vector $\mathbf{a} = [a_1, a_2, \ldots, a_n]$, **chuẩn Euclidean** $\|\mathbf{a}\|$ được tính như sau:

$$ \|\mathbf{a}\| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2} $$

**Độ lớn** này đại diện cho **chiều dài** hoặc **kích thước** của vector trong không gian đa chiều.

---

Hiểu được những nền tảng toán học này giúp đảm bảo tính toán độ tương đồng chính xác, cho phép hiệu suất tốt hơn trong các tác vụ như **tìm kiếm ngữ nghĩa**, **hệ thống truy xuất** và **công cụ đề xuất**. 🚀


----
### Similarity calculation between `embedded_query` and `embedded_document` 
- `embed_documents` : For embedding multiple texts (documents)
- `embed_query` : For embedding a single text (query)

We've implemented a method to search for the most relevant documents using **text embeddings.** 
- Let's use `search_similar_documents(q, docs, hf_embeddings)` to find the most relevant documents.

In [5]:
import numpy as np


def search_similar_documents(q, docs, hf_embeddings):
    """
    Search for the most relevant documents based on a query using text embeddings.

    Args:
        q (str): The query string for which relevant documents are to be found.
        docs (list of str): A list of document strings to compare against the query.
        hf_embeddings: An embedding model object with `embed_query` and `embed_documents` methods.

    Returns:
        tuple:
            - embedded_query (numpy.ndarray): The embedding vector of the query.
            - embedded_documents (numpy.ndarray): The embedding matrix of the documents.

    Workflow:
        1. Embed the query string into a numerical vector using `embed_query`.
        2. Embed each document into numerical vectors using `embed_documents`.
        3. Calculate similarity scores between the query and documents using the dot product.
        4. Sort the documents based on their similarity scores in descending order.
        5. Print the query and display the sorted documents by their relevance.
        6. Return the query and document embeddings for further analysis if needed.
    """
    # Embed the query and documents using the embedding model
    embedded_query = hf_embeddings.embed_query(q)
    embedded_documents = hf_embeddings.embed_documents(docs)

    # Calculate similarity scores using dot product
    similarity_scores = np.array(embedded_query) @ np.array(embedded_documents).T

    # Sort documents by similarity scores in descending order
    sorted_idx = similarity_scores.argsort()[::-1]

    # Display the results
    print(f"[Query] {q}\n" + "=" * 40)
    for i, idx in enumerate(sorted_idx):
        print(f"[{i}] {docs[idx]}")
        print()

    # Return embeddings for potential further processing or analysis
    return embedded_query, embedded_documents

## Tổng quan về HuggingFaceEndpointEmbeddings

**HuggingFaceEndpointEmbeddings** là một tính năng trong thư viện **LangChain** tận dụng **điểm cuối Hugging Face Inference API** để tạo ra các `embeddings` văn bản một cách liền mạch.

---

### 📚 **Các khái niệm chính**

1.  **Hugging Face Inference API**
    -   Truy cập các mô hình `embedding` được đào tạo trước thông qua API của Hugging Face.
    -   Không cần tải xuống mô hình cục bộ; `embeddings` được tạo trực tiếp thông qua API.

2.  **Tích hợp LangChain**
    -   Dễ dàng tích hợp kết quả `embedding` vào quy trình làm việc LangChain bằng giao diện tiêu chuẩn hóa của nó.

3.  **Trường hợp sử dụng**
    -   Tính toán độ tương đồng truy vấn văn bản và tài liệu
    -   Hệ thống tìm kiếm và đề xuất
    -   Các ứng dụng Hiểu ngôn ngữ tự nhiên (NLU)

---

### ⚙️ **Các tham số chính**

-   `model`: ID mô hình Hugging Face (ví dụ: `BAAI/bge-m3`)
-   `task`: Tác vụ cần thực hiện (thường là `"feature-extraction"`)
-   `api_key`: Mã thông báo API Hugging Face của bạn
-   `model_kwargs`: Các tham số cấu hình mô hình bổ sung

---

### 💡 **Ưu điểm**

-   **Không cần tải xuống mô hình cục bộ:** Truy cập tức thì qua API.
-   **Khả năng mở rộng:** Hỗ trợ nhiều mô hình Hugging Face được đào tạo trước.
-   **Tích hợp liền mạch:** Dễ dàng tích hợp `embeddings` vào quy trình làm việc LangChain.

---

### ⚠️ **Lưu ý**

-   **Hỗ trợ API:** Không phải tất cả các mô hình đều hỗ trợ suy luận API.
-   **Tốc độ & Chi phí:** API miễn phí có thể có thời gian phản hồi chậm hơn và giới hạn sử dụng.

---

Với **HuggingFaceEndpointEmbeddings**, bạn có thể dễ dàng tích hợp các mô hình `embedding` mạnh mẽ của Hugging Face vào **quy trình làm việc LangChain** của mình để có các giải pháp NLP hiệu quả và có khả năng mở rộng. 🚀


---
Let’s use the `intfloat/multilingual-e5-large-instruct` model via the API to search for the most relevant documents using text embeddings.

- [intfloat/multilingual-e5-large-instruct](https://huggingface.co/intfloat/multilingual-e5-large-instruct)

In [10]:
from dotenv import load_dotenv
load_dotenv(override=True, dotenv_path="../.env")

True

In [11]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_endpoint_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

Search for the most relevant documents based on a query using text embeddings.

In [12]:
%%time
# Embed the query and documents using the embedding model
embedded_query = hf_endpoint_embeddings.embed_query(q)
embedded_documents = hf_endpoint_embeddings.embed_documents(docs)

CPU times: user 8.07 ms, sys: 1.03 ms, total: 9.1 ms
Wall time: 1.23 s


In [13]:
# Calculate similarity scores using dot product
similarity_scores = np.array(embedded_query) @ np.array(embedded_documents).T

# Sort documents by similarity scores in descending order
sorted_idx = similarity_scores.argsort()[::-1]

In [14]:
# Display the results
print(f"[Query] {q}\n" + "=" * 40)
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {docs[idx]}")
    print()

[Query] Please tell me more about LangChain.
[0] LangChain simplifies the process of building applications with large language models.

[1] LangChain simplifies the process of building applications with large-scale language models.

[2] The LangChain English tutorial is structured based on LangChain's official documentation, cookbook, and various practical examples to help users utilize LangChain more easily and effectively.

[3] Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.

[4] Hi, nice to meet you.



In [15]:
print("[HuggingFace Endpoint Embedding]")
print(f"Model: \t\t{model_name}")
print(f"Document Dimension: \t{len(embedded_documents[0])}")
print(f"Query Dimension: \t{len(embedded_query)}")

[HuggingFace Endpoint Embedding]
Model: 		intfloat/multilingual-e5-large-instruct
Document Dimension: 	1024
Query Dimension: 	1024


Chúng ta có thể xác minh rằng kích thước của `embedded_documents` và `embedded_query` nhất quán.

Bạn cũng có thể thực hiện tìm kiếm bằng phương thức `search_similar_documents` mà chúng ta đã triển khai trước đó.
Từ bây giờ, hãy sử dụng phương thức này cho các tìm kiếm của chúng ta.


In [16]:
%%time
embedded_query, embedded_documents = search_similar_documents(q, docs, hf_endpoint_embeddings)

[Query] Please tell me more about LangChain.
[0] LangChain simplifies the process of building applications with large language models.

[1] LangChain simplifies the process of building applications with large-scale language models.

[2] The LangChain English tutorial is structured based on LangChain's official documentation, cookbook, and various practical examples to help users utilize LangChain more easily and effectively.

[3] Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.

[4] Hi, nice to meet you.

CPU times: user 9.11 ms, sys: 54 μs, total: 9.16 ms
Wall time: 709 ms


## HuggingFaceEmbeddings Overview

- **HuggingFaceEmbeddings** là một tính năng trong thư viện **LangChain** cho phép chuyển đổi dữ liệu văn bản thành vectơ bằng cách sử dụng **Hugging Face embedding models.**
- Lớp này tải xuống và vận hành các mô hình Hugging Face **locally** để xử lý hiệu quả.

---

### 📚 **Các khái niệm chính**

1. **Hugging Face Pre-trained Models**
   - Sử dụng các mô hình embedding được huấn luyện trước (pre-trained) do Hugging Face cung cấp.
   - Tải xuống các mô hình **locally** để thực hiện trực tiếp các hoạt động embedding.

2. **LangChain Integration**
   - Tích hợp liền mạch với quy trình làm việc của LangChain bằng giao diện tiêu chuẩn hóa của nó.

3. **Use Cases**
   - Tính toán độ tương tự giữa truy vấn văn bản và tài liệu
   - Hệ thống tìm kiếm và đề xuất
   - Ứng dụng Natural Language Understanding (NLU)

---

### ⚙️ **Các tham số chính**

- `model_name`: ID mô hình Hugging Face (ví dụ: `sentence-transformers/all-MiniLM-L6-v2`)
- `model_kwargs`: Các tham số cấu hình mô hình bổ sung (ví dụ: cài đặt thiết bị GPU/CPU)
- `encode_kwargs`: Các cài đặt bổ sung cho việc tạo embedding

---

### 💡 **Ưu điểm**

- **Local Embedding Operations:** Thực hiện embedding cục bộ mà không cần kết nối internet.
- **High Performance:** Sử dụng cài đặt GPU để tạo embedding nhanh hơn.
- **Model Variety:** Hỗ trợ nhiều loại mô hình Hugging Face.

---

### ⚠️ **Lưu ý**

- **Local Storage Requirement:** Các mô hình được huấn luyện trước phải được tải xuống cục bộ.
- **Environment Configuration:** Hiệu suất có thể thay đổi tùy thuộc vào cài đặt thiết bị GPU/CPU.

---

Với **HuggingFaceEmbeddings**, bạn có thể sử dụng hiệu quả **Hugging Face's powerful embedding models** trong môi trường **local**, cho phép các giải pháp NLP linh hoạt và có khả năng mở rộng. 🚀


---
Let's download the embedding model locally, perform embeddings, and search for the most relevant documents.

`intfloat/multilingual-e5-large-instruct` 

- [intfloat/multilingual-e5-large-instruct](https://huggingface.co/intfloat/multilingual-e5-large-instruct)

In [17]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings_e5_instruct = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": device},  # mps, cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

sentence_xlm-roberta_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [18]:
%%time
embedded_query, embedded_documents = search_similar_documents(q, docs, hf_embeddings_e5_instruct)

[Query] Please tell me more about LangChain.
[0] LangChain simplifies the process of building applications with large language models.

[1] LangChain simplifies the process of building applications with large-scale language models.

[2] The LangChain English tutorial is structured based on LangChain's official documentation, cookbook, and various practical examples to help users utilize LangChain more easily and effectively.

[3] Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.

[4] Hi, nice to meet you.

CPU times: user 10.6 s, sys: 0 ns, total: 10.6 s
Wall time: 681 ms


In [19]:
print(f"Model: \t\t{model_name}")
print(f"Document Dimension: \t{len(embedded_documents[0])}")
print(f"Query Dimension: \t{len(embedded_query)}")

Model: 		intfloat/multilingual-e5-large-instruct
Document Dimension: 	1024
Query Dimension: 	1024


## FlagEmbedding Usage Guide


- **FlagEmbedding** là một framework embedding tiên tiến được phát triển bởi **BAAI (Beijing Academy of Artificial Intelligence).**
- Nó hỗ trợ **various embedding approaches** và chủ yếu được sử dụng với mô hình **BGE (BAAI General Embedding).**
- FlagEmbedding vượt trội trong các nhiệm vụ như **semantic search**, **natural language processing (NLP)** và **recommendation systems.**

---

### 📚 **Các khái niệm cốt lõi của FlagEmbedding**

1️⃣ `Dense Embedding`
- Định nghĩa: Biểu diễn ý nghĩa tổng thể của một văn bản dưới dạng một vectơ mật độ cao duy nhất.
- Ưu điểm: Nắm bắt hiệu quả sự tương đồng về ngữ nghĩa.
- Trường hợp sử dụng: Tìm kiếm ngữ nghĩa, tính toán độ tương đồng của tài liệu.

2️⃣ `Lexical Embedding`
- Định nghĩa: Phân tách văn bản thành các thành phần cấp từ, nhấn mạnh việc khớp từ.
- Ưu điểm: Đảm bảo khớp chính xác các từ hoặc cụm từ cụ thể.
- Trường hợp sử dụng: Tìm kiếm dựa trên từ khóa, khớp từ chính xác.

3️⃣ `Multi-Vector Embedding`
- Định nghĩa: Chia một tài liệu thành nhiều vectơ để biểu diễn.
- Ưu điểm: Cho phép biểu diễn chi tiết hơn đối với các văn bản dài hoặc các chủ đề đa dạng.
- Trường hợp sử dụng: Phân tích cấu trúc tài liệu phức tạp, khớp chủ đề chi tiết.


---

FlagEmbedding cung cấp một **flexible and powerful toolkit** để tận dụng embeddings trên một loạt các **NLP tasks và semantic search applications.** 🚀

Đoạn mã sau được sử dụng để kiểm soát **tokenizer parallelism** trong thư viện `transformers` của Hugging Face:

- `TOKENIZERS_PARALLELISM = "true"` → **Optimized for speed**, phù hợp cho xử lý dữ liệu quy mô lớn.
- `TOKENIZERS_PARALLELISM = "false"` → **Ensures stability**, ngăn ngừa xung đột và điều kiện chạy đua (race conditions).


In [20]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"  # "false"

In [21]:
# install FlagEmbedding
%pip install -qU FlagEmbedding

Note: you may need to restart the kernel to use updated packages.


### ⚙️ **Tham số chính**

`BGEM3FlagModel`
- `model_name`: **Model ID** của Hugging Face (ví dụ: `BAAI/bge-m3`).
- `use_fp16`: Khi được đặt thành **True**, giảm **memory usage** và cải thiện **encoding speed.**

`bge_embeddings.encode`
- `batch_size`: Xác định **number of documents** cần xử lý cùng một lúc.
- `max_length`: Đặt **maximum token length** cho việc encoding documents.
    - Tăng lên cho các tài liệu dài hơn để đảm bảo mã hóa toàn bộ nội dung.
    - Giá trị quá lớn có thể **degrade performance.**
- `return_dense`: Khi được đặt thành **True**, chỉ trả về **Dense Vectors.**
- `return_sparse`: Khi được đặt thành **True**, trả về **Sparse Vectors.**
- `return_colbert_vecs`: Khi được đặt thành **True**, trả về **ColBERT-style vectors.**

### 1️⃣ **Ví dụ về Dense Vector Embedding**
- Định nghĩa: Biểu diễn ý nghĩa tổng thể của một văn bản dưới dạng một vectơ mật độ cao duy nhất.
- Ưu điểm: Nắm bắt hiệu quả sự tương đồng về ngữ nghĩa.
- Trường hợp sử dụng: Tìm kiếm ngữ nghĩa, tính toán độ tương đồng của tài liệu.


In [22]:
from FlagEmbedding import BGEM3FlagModel

model_name = "BAAI/bge-m3"

bge_embeddings = BGEM3FlagModel(
    model_name,
    use_fp16=True,  # Enabling fp16 improves encoding speed with minimal precision trade-off.
)

# Encode documents with specified parameters
embedded_documents_dense_vecs = bge_embeddings.encode(
    sentences=docs,
    batch_size=12,
    max_length=8192,  # Reduce this value if your documents are shorter to speed up encoding.
)["dense_vecs"]

# Query Encoding
embedded_query_dense_vecs = bge_embeddings.encode(
    sentences=[q],
    batch_size=12,
    max_length=8192,  # Reduce this value if your documents are shorter to speed up encoding.
)["dense_vecs"]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [25]:
embedded_documents_dense_vecs.shape

(5, 1024)

In [26]:
embedded_query_dense_vecs.shape

(1, 1024)

In [27]:
# Calculating Similarity Between Documents and Query
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(
    embedded_query_dense_vecs, embedded_documents_dense_vecs
)
most_similar_idx = similarities.argmax()

# Display the Most Similar Document
print(f"Question: {q}")
print(f"Most similar document: {docs[most_similar_idx]}")

Question: Please tell me more about LangChain.
Most similar document: LangChain simplifies the process of building applications with large language models.


In [28]:
from FlagEmbedding import BGEM3FlagModel

model_name = "BAAI/bge-m3"

bge_embeddings = BGEM3FlagModel(
    model_name,
    use_fp16=True,  # Enabling fp16 improves encoding speed with minimal precision trade-off.
)

# Encode documents with specified parameters
embedded_documents_dense_vecs_default = bge_embeddings.encode(
    sentences=docs, return_dense=True
)["dense_vecs"]

# Query Encoding
embedded_query_dense_vecs_default = bge_embeddings.encode(
    sentences=[q], return_dense=True
)["dense_vecs"]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [29]:
# Calculating Similarity Between Documents and Query
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(
    embedded_query_dense_vecs_default, embedded_documents_dense_vecs_default
)
most_similar_idx = similarities.argmax()

# Display the Most Similar Document
print(f"Question: {q}")
print(f"Most similar document: {docs[most_similar_idx]}")

Question: Please tell me more about LangChain.
Most similar document: LangChain simplifies the process of building applications with large language models.


### 2️⃣ **Sparse(Lexical) Vector Embedding Example**

**Sparse Embedding (Trọng số từ vựng)**
- **Sparse embedding** là một phương pháp embedding sử dụng **high-dimensional vectors where most values are zero.**
- Phương pháp sử dụng **lexical weight** tạo ra embeddings bằng cách xem xét **importance of each word.**

**Cách thức hoạt động**
1.  Tính toán **lexical weight** cho mỗi từ. Các kỹ thuật như **TF-IDF** hoặc **BM25** có thể được sử dụng.
2.  Đối với mỗi từ trong một tài liệu hoặc truy vấn, gán một giá trị cho chiều tương ứng của **sparse vector** dựa trên lexical weight của nó.
3.  Kết quả là, các tài liệu và truy vấn được biểu diễn dưới dạng **high-dimensional vectors where most values are zero.**

**Ưu điểm**
- Phản ánh trực tiếp **importance of words.**
- Cho phép **precise matching** của các từ hoặc cụm từ cụ thể.
- **Faster computation** so với dense embeddings.


In [30]:
from FlagEmbedding import BGEM3FlagModel

model_name = "BAAI/bge-m3"

bge_embeddings = BGEM3FlagModel(
    model_name,
    use_fp16=True,  # Enabling fp16 improves encoding speed with minimal precision trade-off.
)

# Encode documents with specified parameters
embedded_documents_sparse_vecs = bge_embeddings.encode(
    sentences=docs, return_sparse=True
)

# Query Encoding
embedded_query_sparse_vecs = bge_embeddings.encode(sentences=[q], return_sparse=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [31]:
lexical_scores_0 = bge_embeddings.compute_lexical_matching_score(
    embedded_query_sparse_vecs["lexical_weights"][0],
    embedded_documents_sparse_vecs["lexical_weights"][0],
)

lexical_scores_1 = bge_embeddings.compute_lexical_matching_score(
    embedded_query_sparse_vecs["lexical_weights"][0],
    embedded_documents_sparse_vecs["lexical_weights"][1],
)

lexical_scores_2 = bge_embeddings.compute_lexical_matching_score(
    embedded_query_sparse_vecs["lexical_weights"][0],
    embedded_documents_sparse_vecs["lexical_weights"][2],
)

lexical_scores_3 = bge_embeddings.compute_lexical_matching_score(
    embedded_query_sparse_vecs["lexical_weights"][0],
    embedded_documents_sparse_vecs["lexical_weights"][3],
)

lexical_scores_4 = bge_embeddings.compute_lexical_matching_score(
    embedded_query_sparse_vecs["lexical_weights"][0],
    embedded_documents_sparse_vecs["lexical_weights"][4],
)

In [32]:
print(f"question: {q}")
print("====================")
for i, doc in enumerate(docs):
    print(doc, f": {eval(f'lexical_scores_{i}')}")

question: Please tell me more about LangChain.
Hi, nice to meet you. : 0.011874185875058174
LangChain simplifies the process of building applications with large language models. : 0.23139647534117103
The LangChain English tutorial is structured based on LangChain's official documentation, cookbook, and various practical examples to help users utilize LangChain more easily and effectively. : 0.1879164595156908
LangChain simplifies the process of building applications with large-scale language models. : 0.22665631817653775
Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses. : 0.002352734562009573


### 3️⃣ **Multi-Vector(ColBERT) Embedding Example**

**ColBERT** (Contextualized Late Interaction over BERT) là một phương pháp hiệu quả cho **document retrieval.**
- Phương pháp này sử dụng **multi-vector strategy** để biểu diễn cả tài liệu và truy vấn bằng nhiều vectơ.

**Cách thức hoạt động**
1.  Tạo **separate vector** cho mỗi **token in a document**, dẫn đến nhiều vectơ cho mỗi tài liệu.
2.  Tương tự, tạo **separate vector** cho mỗi **token in a query.**
3.  Trong quá trình retrieval, tính toán **similarity** giữa mỗi vectơ token truy vấn và tất cả các vectơ token tài liệu.
4.  Tổng hợp các điểm tương đồng này để tạo ra **final retrieval score.**

**Ưu điểm**
- Cho phép **fine-grained token-level matching.**
- Nắm bắt **contextual embeddings** một cách hiệu quả.
- Hoạt động hiệu quả ngay cả với **long documents.**


In [33]:
from FlagEmbedding import BGEM3FlagModel

model_name = "BAAI/bge-m3"

bge_embeddings = BGEM3FlagModel(
    model_name,
    use_fp16=True,  # Enabling fp16 improves encoding speed with minimal precision trade-off.
)

# Encode documents with specified parameters
embedded_documents_colbert_vecs = bge_embeddings.encode(
    sentences=docs, return_colbert_vecs=True
)

# Query Encoding
embedded_query_colbert_vecs = bge_embeddings.encode(
    sentences=[q], return_colbert_vecs=True
)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [34]:
colbert_scores_0 = bge_embeddings.colbert_score(
    embedded_query_colbert_vecs["colbert_vecs"][0],
    embedded_documents_colbert_vecs["colbert_vecs"][0],
)

colbert_scores_1 = bge_embeddings.colbert_score(
    embedded_query_colbert_vecs["colbert_vecs"][0],
    embedded_documents_colbert_vecs["colbert_vecs"][1],
)

colbert_scores_2 = bge_embeddings.colbert_score(
    embedded_query_colbert_vecs["colbert_vecs"][0],
    embedded_documents_colbert_vecs["colbert_vecs"][2],
)

colbert_scores_3 = bge_embeddings.colbert_score(
    embedded_query_colbert_vecs["colbert_vecs"][0],
    embedded_documents_colbert_vecs["colbert_vecs"][3],
)

colbert_scores_4 = bge_embeddings.colbert_score(
    embedded_query_colbert_vecs["colbert_vecs"][0],
    embedded_documents_colbert_vecs["colbert_vecs"][4],
)

In [35]:
print(f"question: {q}")
print("====================")
for i, doc in enumerate(docs):
    print(doc, f": {eval(f'colbert_scores_{i}')}")

question: Please tell me more about LangChain.
Hi, nice to meet you. : 0.5088493824005127
LangChain simplifies the process of building applications with large language models. : 0.703724205493927
The LangChain English tutorial is structured based on LangChain's official documentation, cookbook, and various practical examples to help users utilize LangChain more easily and effectively. : 0.6633750796318054
LangChain simplifies the process of building applications with large-scale language models. : 0.7055995464324951
Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses. : 0.38072970509529114


### 💡 **Ưu điểm của FlagEmbedding**

-   **Diverse Embedding Options:** Hỗ trợ các phương pháp **Dense, Lexical và Multi-Vector.**
-   **High-Performance Models:** Sử dụng các mô hình được huấn luyện trước mạnh mẽ như **BGE.**
-   **Flexibility:** Chọn phương pháp embedding tối ưu dựa trên **use case** của bạn.
-   **Scalability:** Có khả năng thực hiện embeddings trên **large-scale datasets.**

---

### ⚠️ **Cân nhắc**

-   **Model Size:** Một số mô hình có thể yêu cầu **significant storage capacity.**
-   **Resource Requirements:** **GPU usage is recommended** cho các tính toán vectơ quy mô lớn.
-   **Configuration Needs:** Hiệu suất tối ưu có thể yêu cầu **parameter tuning.**

---

### 📊 **So sánh vectơ FlagEmbedding**

| **Embedding Type** | **Strengths** | **Use Cases** |
|---|---|---|
| **Dense Vector** | Nhấn mạnh sự tương đồng ngữ nghĩa | Semantic search, document matching |
| **Lexical Vector** | Khớp từ chính xác | Keyword search, exact matches |
| **Multi-Vector** | Nắm bắt ý nghĩa phức tạp | Long document analysis, topic classification |
